In [70]:
import cv2
import face_recognition
import csv
import datetime
import os

In [71]:
#taking image from csv file
def load_known_faces(csv_file):
    known_faces = []
    known_names = []

    with open(csv_file, 'r') as file:
        reader = csv.reader(file)
        next(reader)  # Skip header row

        for row in reader:
            name, image_path = row
            known_names.append(name)
            image = face_recognition.load_image_file(image_path)
            face_encoding = face_recognition.face_encodings(image)[0]
            known_faces.append(face_encoding)

    return known_faces, known_names      

In [72]:
# csv_read_path = 'data.csv'
# loaded_known_faces ,loaded_known_names  = load_known_faces(csv_read_path)

# print("Loaded Known Faces:")
# print(loaded_known_faces)
# print("\nLoaded Known Names:")
# print(loaded_known_names)

In [73]:
# # writing to csv
# def write_to_csv(name, current_time):
#     if name != "Unknown":
#         csv_file_path = 'attendance.csv'

#         is_empty = os.stat(csv_file_path).st_size == 0

#         with open(csv_file_path, 'a', newline='') as file:
#             writer = csv.writer(file)

#             if is_empty:
#                 writer.writerow(['Name', 'Time'])

#             writer.writerow([name, current_time])   

# # Example usage:
# name = "John Doe"
# current_time = datetime.datetime.now().strftime('%H:%M:%S')


# write_to_csv(name, current_time)

In [84]:
# writing to csv without duplicates for a specific date

def write_to_csv(name, current_time, current_date):
    if name != "Unknown":
        csv_file_path = 'attendance.csv'

        # Check if the data already exists in the CSV for the current date
        with open(csv_file_path, 'r') as file:
            reader = csv.reader(file)
            next(reader, None)  # Skip the header row
            existing_data = any(len(row) >= 3 and row[0] == name and row[2] == current_date for row in reader)

        # If data doesn't exist, append it
        if not existing_data:
            with open(csv_file_path, 'a', newline='') as file:
                writer = csv.writer(file)

                # If the file is empty, write the header
                if os.stat(csv_file_path).st_size == 0:
                    writer.writerow(['Name', 'Time', 'Date'])

                writer.writerow([name, current_time, current_date])

# # Example :
# name = "Apple"
# current_time = datetime.datetime.now().strftime('%H:%M:%S')
# current_date = datetime.datetime.now().strftime('%Y-%m-%d')

# write_to_csv(name, current_time, current_date)


In [76]:
known_face_encodings, known_face_names = load_known_faces('data.csv')

In [77]:
video_capture = cv2.VideoCapture(0)

recognized_set = set()

while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()

    # Find all face locations and encodings in the current frame
    face_locations = face_recognition.face_locations(frame)
    face_encodings = face_recognition.face_encodings(frame, face_locations)

    recognized_names = []

    for face_encoding in face_encodings:
        # Compare face encodings with known faces
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        name = "Unknown"

        if True in matches:
            matched_index = matches.index(True)
            name = known_face_names[matched_index]

        recognized_names.append(name)

    for name in recognized_names:
        if name not in recognized_set:
            current_time = datetime.datetime.now().strftime('%H:%M:%S')
            current_date = datetime.datetime.now().strftime('%Y-%m-%d')
            write_to_csv(name, current_time, current_date)
            recognized_set.add(name)

    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, recognized_names):
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        cv2.putText(frame, name, (left + 6, bottom - 6), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), 1)

    # Display the resulting frame
    cv2.imshow('Video', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close all windows
video_capture.release()
cv2.destroyAllWindows()